In [1]:
import pandas as pd
import numpy as np
from numpy import meshgrid

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

dfs = pd.read_excel("Acme.xlsx")

dfs_copy = dfs.copy()
dfs_click_copy = dfs.loc[dfs['click'] == 1,].copy()

# Need different X's and y's for the different probabilities.
X1 = dfs_copy[['currently_insured', 'number_of_vehicles', 'number_of_drivers']]
y1 = dfs_copy['rank']

X2 = dfs_click_copy[['currently_insured', 'number_of_vehicles', 'number_of_drivers','rank']]
y2 = dfs_click_copy['policies sold']

X3 = dfs_copy[['currently_insured', 'number_of_vehicles', 'number_of_drivers','rank']]
y3 = dfs_copy['click']

cols = ['currently_insured','number_of_vehicles','number_of_drivers','rank']

X1[['currently_insured','number_of_vehicles','number_of_drivers']] = X1[['currently_insured','number_of_vehicles','number_of_drivers']].astype(str)
X1_prime = X1[['currently_insured','number_of_vehicles','number_of_drivers']]
X2[cols] = X2[cols].astype(str)
X2_prime = X2[cols]
X3[cols] = X3[cols].astype(str)
X3_prime = X3[cols]

X1_prime.loc[X1_prime.currently_insured == "Y",'currently_insured'] = 1
X1_prime.loc[X1_prime.currently_insured == "N",'currently_insured'] = 0

X2_prime.loc[X2_prime.currently_insured == "Y",'currently_insured'] = 1
X2_prime.loc[X2_prime.currently_insured == "N",'currently_insured'] = 0

X3_prime.loc[X3_prime.currently_insured == "Y",'currently_insured'] = 1
X3_prime.loc[X3_prime.currently_insured == "N",'currently_insured'] = 0

y1_prime = y1.astype(str)
y2_prime = y2.astype(str)
y3_prime = y3.astype(str)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

svc = SVC(kernel = 'linear', probability = True)                   # linear kernel seems to work best for SVC.
rfc = RandomForestClassifier(max_depth = 10,n_estimators = 500)
lda = LinearDiscriminantAnalysis()


c:\python3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [2]:

    
svc.fit(X1_prime,y1_prime.ravel())
        
y_predict1 = svc.predict_proba(X1_prime)     #???

lda.fit(X2_prime,y2_prime.ravel())
        
y_predict2 = lda.predict_proba(X2_prime)     #???

rfc.fit(X3_prime,y3_prime.ravel())
        
y_predict3 = rfc.predict_proba(X3_prime)     #???


print(np.shape(y_predict1))
print()
print(np.shape(y_predict2))
print()
print(np.shape(y_predict3))

(10000, 5)

(1374, 2)

(10000, 2)


In [3]:
p1 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "N") & (dfs_copy["number_of_vehicles"] == 1) & (dfs_copy["number_of_drivers"] == 1),'rank'].count())/10000
p2 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "N") & (dfs_copy["number_of_vehicles"] == 2) & (dfs_copy["number_of_drivers"] == 1),'rank'].count())/10000
p3 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "N") & (dfs_copy["number_of_vehicles"] == 1) & (dfs_copy["number_of_drivers"] == 2),'rank'].count())/10000
p4 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "N") & (dfs_copy["number_of_vehicles"] == 2) & (dfs_copy["number_of_drivers"] == 2),'rank'].count())/10000

p5 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "Y") & (dfs_copy["number_of_vehicles"] == 1) & (dfs_copy["number_of_drivers"] == 1),'rank'].count())/10000
p6 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "Y") & (dfs_copy["number_of_vehicles"] == 2) & (dfs_copy["number_of_drivers"] == 1),'rank'].count())/10000
p7 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "Y") & (dfs_copy["number_of_vehicles"] == 1) & (dfs_copy["number_of_drivers"] == 2),'rank'].count())/10000
p8 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "Y") & (dfs_copy["number_of_vehicles"] == 2) & (dfs_copy["number_of_drivers"] == 2),'rank'].count())/10000

def constraint(x1,x2,x3,x4,x5,x6,x7,x8):
    
    d = {'currently_insured': [0], 'number_of_vehicles': [1], 'number_of_drivers':[1], 'rank':[x1]}
    y_predict2_1 = lda.predict_proba(pd.DataFrame(data = d))[0,1]
    y_predict3_1 = rfc.predict_proba(pd.DataFrame(data = d))[0,1]
    d = {'currently_insured': [0], 'number_of_vehicles': [2], 'number_of_drivers':[1], 'rank':[x2]}
    y_predict2_2 = lda.predict_proba(pd.DataFrame(data = d))[0,1]
    y_predict3_2 = rfc.predict_proba(pd.DataFrame(data = d))[0,1]
    d = {'currently_insured': [0], 'number_of_vehicles': [1], 'number_of_drivers':[2], 'rank':[x3]}
    y_predict2_3 = lda.predict_proba(pd.DataFrame(data = d))[0,1]
    y_predict3_3 = rfc.predict_proba(pd.DataFrame(data = d))[0,1]
    d = {'currently_insured': [0], 'number_of_vehicles': [2], 'number_of_drivers':[2], 'rank':[x4]}
    y_predict2_4 = lda.predict_proba(pd.DataFrame(data = d))[0,1]
    y_predict3_4 = rfc.predict_proba(pd.DataFrame(data = d))[0,1]
   
    d = {'currently_insured': [1], 'number_of_vehicles': [1], 'number_of_drivers':[1], 'rank':[x5]}
    y_predict2_5 = lda.predict_proba(pd.DataFrame(data = d))[0,1]
    y_predict3_5 = rfc.predict_proba(pd.DataFrame(data = d))[0,1]
    d = {'currently_insured': [1], 'number_of_vehicles': [2], 'number_of_drivers':[1], 'rank':[x6]}
    y_predict2_6 = lda.predict_proba(pd.DataFrame(data = d))[0,1]
    y_predict3_6 = rfc.predict_proba(pd.DataFrame(data = d))[0,1]
    d = {'currently_insured': [1], 'number_of_vehicles': [1], 'number_of_drivers':[2], 'rank':[x7]}
    y_predict2_7 = lda.predict_proba(pd.DataFrame(data = d))[0,1]
    y_predict3_7 = rfc.predict_proba(pd.DataFrame(data = d))[0,1]
    d = {'currently_insured': [1], 'number_of_vehicles': [2], 'number_of_drivers':[2], 'rank':[x8]}
    y_predict2_8 = lda.predict_proba(pd.DataFrame(data = d))[0,1]
    y_predict3_8 = rfc.predict_proba(pd.DataFrame(data = d))[0,1]
   
    return p1*y_predict2_1*y_predict3_1 + p2*y_predict2_2*y_predict3_2 + p3*y_predict2_3*y_predict3_3 + p4*y_predict2_4*y_predict3_4 + p5*y_predict2_5*y_predict3_5 + p6*y_predict2_6*y_predict3_6 + p7*y_predict2_7*y_predict3_7 + p8*y_predict2_8*y_predict3_8 

In [4]:
p1 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "N") & (dfs_copy["number_of_vehicles"] == 1) & (dfs_copy["number_of_drivers"] == 1),'rank'].count())/10000
p2 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "N") & (dfs_copy["number_of_vehicles"] == 2) & (dfs_copy["number_of_drivers"] == 1),'rank'].count())/10000
p3 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "N") & (dfs_copy["number_of_vehicles"] == 1) & (dfs_copy["number_of_drivers"] == 2),'rank'].count())/10000
p4 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "N") & (dfs_copy["number_of_vehicles"] == 2) & (dfs_copy["number_of_drivers"] == 2),'rank'].count())/10000

p5 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "Y") & (dfs_copy["number_of_vehicles"] == 1) & (dfs_copy["number_of_drivers"] == 1),'rank'].count())/10000
p6 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "Y") & (dfs_copy["number_of_vehicles"] == 2) & (dfs_copy["number_of_drivers"] == 1),'rank'].count())/10000
p7 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "Y") & (dfs_copy["number_of_vehicles"] == 1) & (dfs_copy["number_of_drivers"] == 2),'rank'].count())/10000
p8 = (dfs_copy.loc[(dfs_copy["currently_insured"]== "Y") & (dfs_copy["number_of_vehicles"] == 2) & (dfs_copy["number_of_drivers"] == 2),'rank'].count())/10000
  
    
d1 = {'currently_insured': [0], 'number_of_vehicles': [1], 'number_of_drivers':[1]}
y_predict1_1 = svc.predict_proba(pd.DataFrame(data = d1))[0]
    
d1 = {'currently_insured': [0], 'number_of_vehicles': [2], 'number_of_drivers':[1]}
y_predict1_2 = svc.predict_proba(pd.DataFrame(data = d1))[0]

d1 = {'currently_insured': [0], 'number_of_vehicles': [1], 'number_of_drivers':[2]}
y_predict1_3 = svc.predict_proba(pd.DataFrame(data = d1))[0]

d1 = {'currently_insured': [0], 'number_of_vehicles': [2], 'number_of_drivers':[2]}
y_predict1_4 = svc.predict_proba(pd.DataFrame(data = d1))[0]


d1 = {'currently_insured': [1], 'number_of_vehicles': [1], 'number_of_drivers':[1]}
y_predict1_5 = svc.predict_proba(pd.DataFrame(data = d1))[0]

d1 = {'currently_insured': [1], 'number_of_vehicles': [2], 'number_of_drivers':[1]}
y_predict1_6 = svc.predict_proba(pd.DataFrame(data = d1))[0]

d1 = {'currently_insured': [1], 'number_of_vehicles': [1], 'number_of_drivers':[2]}
y_predict1_7 = svc.predict_proba(pd.DataFrame(data = d1))[0]

d1 = {'currently_insured': [1], 'number_of_vehicles': [2], 'number_of_drivers':[2]}
y_predict1_8 = svc.predict_proba(pd.DataFrame(data = d1))[0]



# current ranks:
x1c = np.argmax(y_predict1_1)+1
x2c = np.argmax(y_predict1_2)+1
x3c = np.argmax(y_predict1_3)+1
x4c = np.argmax(y_predict1_4)+1
x5c = np.argmax(y_predict1_5)+1
x6c = np.argmax(y_predict1_6)+1
x7c = np.argmax(y_predict1_7)+1
x8c = np.argmax(y_predict1_8)+1

def weight(x1,x2,x3,x4,x5,x6,x7,x8):
    
    if (x1-x1c != 0):
        a1 = p1*(x1/5)*1/np.abs(x1-x1c)
    else:
        a1 = p1*(x1/5)
        
    if (x2-x2c != 0):
        a2 = p2*(x2/5)*1/np.abs(x2-x2c)
    else:
        a2 = p2*(x2/5)
        
    if (x3-x3c != 0):
        a3 = p3*(x3/5)*1/np.abs(x3-x3c)
    else:
        a3 = p3*(x3/5)
        
    if (x4-x4c != 0):
        a4 = p4*(x4/5)*1/np.abs(x4-x4c)
    else:
        a4 = p4*(x4/5)
        
    if (x5-x5c != 0):
        a5 = p5*(x5/5)*1/np.abs(x5-x5c)
    else:
        a5 = p5*(x5/5)
        
    if (x6-x6c != 0):
        a6 = p6*(x6/5)*1/np.abs(x6-x6c)
    else:
        a6 = p6*(x6/5)
        
    if (x7-x7c != 0):
        a7 = p7*(x7/5)*1/np.abs(x7-x7c)
    else:
        a7 = p7*(x7/5)
        
    if (x8-x8c != 0):
        a8 = p8*(x8/5)*1/np.abs(x8-x8c)
    else:
        a8 = p8*(x8/5)
        

    
    return a1 + a2 + a3 + a4 + a5 + a6 + a7 + a8

In [5]:
d = {'currently_insured': [1], 'number_of_vehicles': [2], 'number_of_drivers':[2], 'rank':[1]}
d1 = {'currently_insured': [1], 'number_of_vehicles': [2], 'number_of_drivers':[2]}
y_predict1_1 = svc.predict_proba(pd.DataFrame(data = d1))
y_predict2_1 = lda.predict_proba(pd.DataFrame(data = d))
y_predict3_1 = rfc.predict_proba(pd.DataFrame(data = d))

print(y_predict1_1)
print(y_predict2_1)
print(y_predict3_1)
print(p1)


[[0.00849591 0.07712704 0.32625002 0.44626888 0.14185814]]
[[0.79301759 0.20698241]]
[[0.6081058 0.3918942]]
0.0785


In [6]:
# As we can see above, y_predict1 is our rank probability prediction, given type

# y_predict2 is our policy sold probability prediction, given click = 1 and rank = x, type = t
# y_predict3 is our click == 1 probability prediction, given rank = x and type = t.

# itertools.combinations_with_replacement(iterable, r)???

minweight = weight(x1c,x2c,x3c,x4c,x5c,x6c,x7c,x8c)        # weight of original strategy
rankdiff = 1     # this is how much we allow our new ranks (x1,...,x8) to differ from original ones.

for x1 in [1,2,3,4,5]:
    
    if np.abs(x1-x1c) <= rankdiff:
        for x2 in [1,2,3,4,5]:
        
            if np.abs(x2-x2c) <= rankdiff:
                for x3 in [1,2,3,4,5]:
            
                    if np.abs(x3-x3c) <= rankdiff:
                        for x4 in [1,2,3,4,5]:
                
                            if np.abs(x4-x4c) <= rankdiff:
                                for x5 in [1,2,3,4,5]:
                    
                                    if np.abs(x5-x5c) <= rankdiff:
                                        for x6 in [1,2,3,4,5]:
                        
                                            if np.abs(x6-x6c) <= rankdiff:
                                                for x7 in [1,2,3,4,5]:
                            
                                                    if np.abs(x7-x7c) <= rankdiff:
                                                        for x8 in [1,2,3,4,5]:
                                
                                                            if np.abs(x8-x8c) <= rankdiff:                                           
                                                                if constraint(x1,x2,x3,x4,x5,x6,x7,x8) >= 0.04:
                                                                    if weight(x1,x2,x3,x4,x5,x6,x7,x8) >= minweight:
            
                                                                        bestranks = [x1,x2,x3,x4,x5,x6,x7,x8]
                                                                        minweight = weight(x1,x2,x3,x4,x5,x6,x7,x8)
        
    

    
    

In [16]:
print("We have removed marital_status so that the code can actually run in time.")
print("Recommended new ranks: ", bestranks)
print("These correspond to the following types:")
print("type (i=0, v=1, d=1) gets new rank: ", bestranks[0])
print("type (i=0, v=2, d=1) gets new rank: ", bestranks[1])
print("type (i=0, v=1, d=2) gets new rank: ", bestranks[2])
print("type (i=0, v=2, d=2) gets new rank: ", bestranks[3])
print("type (i=1, v=1, d=1) gets new rank: ", bestranks[4])
print("type (i=1, v=2, d=1) gets new rank: ", bestranks[5])
print("type (i=1, v=1, d=2) gets new rank: ", bestranks[6])
print("type (i=1, v=2, d=2) gets new rank: ", bestranks[7])
print()
print("Old ranks:")
print("type (i=0, v=1, d=1) has old rank: ", x1c)
print("type (i=0, v=2, d=1) has old rank: ", x2c)
print("type (i=0, v=1, d=2) has old rank: ", x3c)
print("type (i=0, v=2, d=2) has old rank: ", x4c)
print("type (i=1, v=1, d=1) has old rank: ", x5c)
print("type (i=1, v=2, d=1) has old rank: ", x6c)
print("type (i=1, v=1, d=2) has old rank: ", x7c)
print("type (i=1, v=2, d=2) has old rank: ", x8c)

print()
print("Recall that higher weight is better in terms of cost and maximum weight is 1.")
print("Weight of new strategy: ", np.round(minweight,5))
print("Weight of original strategy: ", np.round(weight(x1c,x2c,x3c,x4c,x5c,x6c,x7c,x8c),5))

We have removed marital_status so that the code can actually run in time.
Recommended new ranks:  [1, 5, 2, 5, 4, 5, 3, 5]
These correspond to the following types:
type (i=0, v=1, d=1) gets new rank:  1
type (i=0, v=2, d=1) gets new rank:  5
type (i=0, v=1, d=2) gets new rank:  2
type (i=0, v=2, d=2) gets new rank:  5
type (i=1, v=1, d=1) gets new rank:  4
type (i=1, v=2, d=1) gets new rank:  5
type (i=1, v=1, d=2) gets new rank:  3
type (i=1, v=2, d=2) gets new rank:  5

Old ranks:
type (i=0, v=1, d=1) has old rank:  1
type (i=0, v=2, d=1) has old rank:  5
type (i=0, v=1, d=2) has old rank:  1
type (i=0, v=2, d=2) has old rank:  4
type (i=1, v=1, d=1) has old rank:  3
type (i=1, v=2, d=1) has old rank:  5
type (i=1, v=1, d=2) has old rank:  2
type (i=1, v=2, d=2) has old rank:  4

Recall that higher weight is better in terms of cost and maximum weight is 1.
Weight of new strategy:  0.80382
Weight of original strategy:  0.70498
